In [1]:
import pandas as pd
import numpy as np
from datetime import *
from glob import glob

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.4.0


In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score

In [4]:
import pylab as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## Get data

In [5]:
with open('pipeline_data/steps.dat', 'r') as handle:
    steps = handle.readline().strip().split()

In [6]:
csv_fname = glob('pipeline_data/data_????-??-??_glm.csv')[0]
df = pd.read_csv(csv_fname, dtype={'fips': str}).set_index('fips')
df.head()

case0  case1  case2  case3  case4  case5  case6  case7  case8  case9  \
fips                                                                          
01001   12.0    7.0    6.0   11.0   11.0   16.0   45.0   48.0   60.0   48.0   
01003   34.0   36.0   40.0   45.0   30.0   34.0   42.0   19.0   12.0   29.0   
01005    3.0    7.0    8.0   16.0    9.0   14.0   24.0   27.0   47.0   41.0   
01007    4.0    9.0   13.0    8.0    7.0    4.0    7.0    7.0   15.0    5.0   
01009   12.0    3.0    9.0   12.0    6.0    5.0    2.0    3.0   13.0   11.0   

       ...  death33_glm  death34_glm  death35_glm  death36_glm  death37_glm  \
fips   ...                                                                    
01001  ...         3.32         2.95         4.44         4.85         5.00   
01003  ...         5.28         5.25         7.80         8.73         9.03   
01005  ...         0.86         0.66         0.98         1.02         1.21   
01007  ...         1.33         1.13         1.67         1.77         1.91   
01009  ...         1.22         1.17         1.64         1.77         1.83   

       death38_glm  death39_glm  death40_glm  death41_glm  total_death_glm  
fips                                                                        
01001         4.48         4.88         6.20         6.33            93.11  
01003         7.65         8.82        11.07        11.58           134.33  
01005         1.38         1.43         2.22         2.62            56.28  
01007         1.74         1.90         2.49         2.54            37.07  
01009         1.50         1.73         2.32         2.42            21.42  

[5 rows x 182 columns]

## Functions

In [7]:
class tfRegr:
    
    def __init__(self, 
                 epoch=100, 
                 verbose=False, 
                 validation_split=0.2, 
                 learning_rate=.1):
        
        self.normalizer = preprocessing.Normalization()
        self.ep = epoch
        self.vb = verbose
        self.vs = validation_split
        self.lr = learning_rate
    
    def fit(self, X, y):
        self.normalizer.adapt(X)

        self.model = tf.keras.Sequential([
            self.normalizer,
            layers.Dense(units=1)
        ])

        self.model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=self.lr),
            loss='mean_absolute_error'
        )

        self.history = self.model.fit(
            X, y,
            epochs=self.ep,
            verbose=self.vb, # logging
            validation_split=self.vs # Calculate validation results on a fraction of the training data
        )
        return self.model

In [8]:
def get_prd(x, c=.5, gamma=0.000065):
    prd = (x['ET'] + x['RF'] + c * x['TF']) / (2 + c) 
    arr = prd.diff().fillna(0).values
    panel = np.array([np.exp(-gamma * a) for a in arr])
    return prd * panel

In [9]:
def predict(df, cur_idx):

    regrs = {
        'RF': RandomForestRegressor(min_samples_split=2), 
        'ET': ExtraTreesRegressor(min_samples_split=2), 
        'TF': tfRegr(),
    }

    glm = [
        f'death{cur_idx - 1}_glm', 
        f'death{cur_idx}_glm', 
        f'case{cur_idx - 1}_glm', 
        f'case{cur_idx}_glm'
    ] 
    cols = [f'death{cur_idx - 1}', f'case{cur_idx - 1}'] + glm
    X, y = df[cols].values, df[f'death{cur_idx}'].values 
    
    # ========================= get case training data ========================START
    cols = [f'case{cur_idx - 1}', f'case{cur_idx - 1}_glm', f'case{cur_idx}_glm']
    case_X, case_y = df[cols].values, df[f'case{cur_idx}'].values 
    # ========================= get case training data ========================END
    
    for name in regrs:
        if name == 'TF':
            case_regr = tfRegr()
            death_regr = tfRegr()
        elif name == 'RF':
            case_regr = RandomForestRegressor(min_samples_split=2)
            death_regr = RandomForestRegressor(min_samples_split=2)
        else:
            case_regr = ExtraTreesRegressor(min_samples_split=2)
            death_regr = ExtraTreesRegressor(min_samples_split=2)
        
        # ========================= get case prediction ========================START
        case_regr = case_regr.fit(case_X, case_y)
        cols = [f'case{cur_idx}', f'case{cur_idx - 1}_glm', f'case{cur_idx}_glm']
        df[f'case{cur_idx}_{name}_1'] = case_regr.predict(df[cols].values).flatten()
        # ========================= get case prediction ========================END
        
        death_regr = death_regr.fit(X, y)

        #================== Evaluation =================START
        y_pred = death_regr.predict(X).flatten()
        r2 = r2_score(y, y_pred)
        rmse = np.sqrt(((y - y_pred) ** 2).mean())
        print(f'\t{name}: r2 = {r2:.3f}, rmse = {rmse:.3f}')
        #================== Evaluation =================END
        
        cols = [f'death{cur_idx}', f'case{cur_idx}'] + glm
        df[f'death{cur_idx}_{name}_1'] = death_regr.predict(df[cols].values).flatten()
        
        cols = [f'death{cur_idx}_{name}_1', f'case{cur_idx}_{name}_1'] + glm
        df[f'death{cur_idx}_{name}_2'] = death_regr.predict(df[cols].values).flatten()
    
    prd_cols_1 = [col for col in df.columns if col.startswith('death') and col.endswith('_1')]
    prd_cols_2 = [col for col in df.columns if col.startswith('death') and col.endswith('_2')]
    
    c, gamma = .5, 0.0001
    tmp = df[prd_cols_1].rename(columns={f'death{cur_idx}_{name}_1': name for name in regrs})
    prd_1 = (tmp['ET'] + tmp['RF'] + c * tmp['TF']) / (2 + c) 
    tmp = df[prd_cols_2].rename(columns={f'death{cur_idx}_{name}_2': name for name in regrs})
    prd_2 = (tmp['ET'] + tmp['RF'] + c * tmp['TF']) / (2 + c)
    
    tmp = pd.concat([df[f'death{cur_idx}'], prd_1, prd_2], axis=1)
    panel = tmp.diff(axis=1).dropna(axis=1).applymap(lambda x: np.exp(-gamma * x))
    tmp = tmp[[0, 1]] * panel
    
    cols_to_drop = [col for col in df.columns if col.endswith('_1') or col.endswith('_2')]
    df.drop(cols_to_drop, axis=1, inplace=True)
    
    return tmp.values.T

## Forecast Runs

In [10]:
cur_idx = len(steps) - 1

num_runs = 100
prds_1, prds_2 = [], []

for r in range(num_runs):
    print(f'run = {r}')
    prd_1, prd_2 = predict(df, cur_idx)
    prds_1.append(prd_1)
    prds_2.append(prd_2)
    # print(prd_1.min(), prd_1.max(), prd_1.mean())
    # print(prd_2.min(), prd_2.max(), prd_2.mean())

run = 0
	RF: r2 = 0.946, rmse = 8.614
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.954, rmse = 7.964
run = 1
	RF: r2 = 0.939, rmse = 9.193
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.940, rmse = 9.106
run = 2
	RF: r2 = 0.960, rmse = 7.475
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.944, rmse = 8.789
run = 3
	RF: r2 = 0.941, rmse = 9.051
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.942, rmse = 8.972
run = 4
	RF: r2 = 0.940, rmse = 9.086
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.941, rmse = 9.027
run = 5
	RF: r2 = 0.950, rmse = 8.368
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.950, rmse = 8.337
run = 6
	RF: r2 = 0.936, rmse = 9.398
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.950, rmse = 8.346
run = 7
	RF: r2 = 0.952, rmse = 8.184
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.949, rmse = 8.376
run = 8
	RF: r2 = 0.948, rmse = 8.501
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.951, rmse = 8.275
run = 9
	RF: r2 = 0.944, rmse = 8.785
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.948, rmse = 8.491
run = 10
	RF: r2 = 0

	RF: r2 = 0.957, rmse = 7.730
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.948, rmse = 8.513
run = 84
	RF: r2 = 0.950, rmse = 8.349
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.946, rmse = 8.629
run = 85
	RF: r2 = 0.929, rmse = 9.945
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.950, rmse = 8.311
run = 86
	RF: r2 = 0.947, rmse = 8.583
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.948, rmse = 8.532
run = 87
	RF: r2 = 0.950, rmse = 8.358
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.936, rmse = 9.416
run = 88
	RF: r2 = 0.946, rmse = 8.650
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.953, rmse = 8.065
run = 89
	RF: r2 = 0.952, rmse = 8.179
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.952, rmse = 8.124
run = 90
	RF: r2 = 0.936, rmse = 9.413
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.951, rmse = 8.204
run = 91
	RF: r2 = 0.932, rmse = 9.743
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.951, rmse = 8.227
run = 92
	RF: r2 = 0.947, rmse = 8.557
	ET: r2 = 1.000, rmse = 0.000
	TF: r2 = 0.945, rmse = 8.707
run = 93
	RF: r2 = 

## Prepare for submission

In [11]:
# target end dates
cur_dt = datetime.strptime(steps[cur_idx], '%Y-%m-%d')
fc_dt = (cur_dt + timedelta(days=1)).date()
nxt_dt_1 = (cur_dt + timedelta(days=7)).date()
nxt_dt_2 = (cur_dt + timedelta(days=14)).date()
nxt_dts = [nxt_dt_1, nxt_dt_2]

# quantiles
qs = [.025, .1, .25, .5, .75, .9, .975]

In [12]:
data = np.array(prds_1).T
data[data < 0] = 0
df_county_prd_1 = pd.DataFrame(data=data, index=df.index, columns=range(num_runs))
data = np.array(prds_2).T
data[data < 0] = 0
df_county_prd_2 = pd.DataFrame(data=data, index=df.index, columns=range(num_runs))
county_dfs = [df_county_prd_1, df_county_prd_2]

### County

In [13]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    csv_fname = f'results/{steps[cur_idx]}_county_death_{w}-wk_{num_runs}.csv'
    county_df.to_csv(csv_fname, float_format='%.2f')
    
    # ========================= Point esitmate =========================START
    df_point = county_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'fips': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = county_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['fips']).sort_values('fips')\
        .rename(columns={'fips': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc death'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_county = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
print(df_forecast_county.shape)
df_forecast_county.head()

(49504, 6)


location      value   type quantile                target target_end_date
0    01001   3.902751  point       NA  1 wk ahead inc death      2021-01-23
1    01003  11.419530  point       NA  1 wk ahead inc death      2021-01-23
2    01005   1.478328  point       NA  1 wk ahead inc death      2021-01-23
3    01007   0.754866  point       NA  1 wk ahead inc death      2021-01-23
4    01009   5.162475  point       NA  1 wk ahead inc death      2021-01-23

### States

In [14]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    
    df_tmp = county_df.reset_index()
    df_tmp['state'] = df_tmp['fips'].apply(lambda x: x[:2])
    state_df = df_tmp.groupby(by='state').sum()
    
    # ========================= Point esitmate =========================START
    df_point = state_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'state': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = state_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['state']).sort_values('state')\
        .rename(columns={'state': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc death'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_state = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
df_forecast_state.head()

location        value   type quantile                target target_end_date
0       01   520.227043  point       NA  1 wk ahead inc death      2021-01-23
1       02    15.201267  point       NA  1 wk ahead inc death      2021-01-23
2       04  1037.440956  point       NA  1 wk ahead inc death      2021-01-23
3       05   257.532452  point       NA  1 wk ahead inc death      2021-01-23
4       06  3650.817716  point       NA  1 wk ahead inc death      2021-01-23

### Nation

In [15]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    
    nation_df = pd.DataFrame(data=county_df.sum(axis=0), columns=['US']).T
    nation_df.index.name = 'nation'
    
    # ========================= Point esitmate =========================START
    df_point = nation_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'nation': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = nation_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['nation']).sort_values('nation')\
        .rename(columns={'nation': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc death'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_nation = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
df_forecast_nation

location         value      type quantile                target  \
0        US  22109.326689     point       NA  1 wk ahead inc death   
1        US  21881.405279  quantile    0.025  1 wk ahead inc death   
2        US  21946.758707  quantile      0.1  1 wk ahead inc death   
3        US  22011.739625  quantile     0.25  1 wk ahead inc death   
4        US  22099.064249  quantile      0.5  1 wk ahead inc death   
5        US  22184.808940  quantile     0.75  1 wk ahead inc death   
6        US  22284.917344  quantile      0.9  1 wk ahead inc death   
7        US  22402.599006  quantile    0.975  1 wk ahead inc death   
8        US  20412.012280     point       NA  2 wk ahead inc death   
9        US  20060.167069  quantile    0.025  2 wk ahead inc death   
10       US  20163.596443  quantile      0.1  2 wk ahead inc death   
11       US  20275.685744  quantile     0.25  2 wk ahead inc death   
12       US  20409.612054  quantile      0.5  2 wk ahead inc death   
13       US  20532.360703  quantile     0.75  2 wk ahead inc death   
14       US  20691.799952  quantile      0.9  2 wk ahead inc death   
15       US  20827.267562  quantile    0.975  2 wk ahead inc death   

   target_end_date  
0       2021-01-23  
1       2021-01-23  
2       2021-01-23  
3       2021-01-23  
4       2021-01-23  
5       2021-01-23  
6       2021-01-23  
7       2021-01-23  
8       2021-01-30  
9       2021-01-30  
10      2021-01-30  
11      2021-01-30  
12      2021-01-30  
13      2021-01-30  
14      2021-01-30  
15      2021-01-30

In [16]:
df_forecast = pd.concat([df_forecast_state, df_forecast_nation])
df_forecast.reset_index(drop=True, inplace=True)
df_forecast['forecast_date'] = fc_dt
cols = ['forecast_date', 'target', 'target_end_date', 'location', 'type', 'quantile', 'value']
df_forecast = df_forecast[cols]

# =============== make nagative value zero ==============
value_dezero = df_forecast['value']
value_dezero[value_dezero < 0]  = 0
df_forecast['value'] = value_dezero

In [17]:
team = 'UChicagoCHATTOPADHYAY'
model = 'UnIT'
death_csv = f'results/{fc_dt}-{team}-{model}_death.csv'
df_forecast.to_csv(death_csv, float_format='%.2f', index=False)

## Validation

In [18]:
validation = '/home/yihuang/Documents/Data/covid19-forecast-hub/code/validation/validate_single_forecast_file.py'

! python3 {validation} {death_csv}

/Library/Frameworks/Python.framework/Versions/3.9/bin/python3: can't open file '/home/yihuang/Documents/Data/covid19-forecast-hub/code/validation/validate_single_forecast_file.py': [Errno 2] No such file or directory


In [19]:
case_csv = f'results/{fc_dt}-UChicagoCHATTOPADHYAY-UnIT_case.csv'
df_forecast_case = pd.read_csv(case_csv, dtype={'location': str})

df_combined = pd.concat([df_forecast_case, df_forecast], axis=0)
combined_csv = f'submission/{fc_dt}-UChicagoCHATTOPADHYAY-UnIT.csv'
df_combined.to_csv(combined_csv, index=False)

! python3 {validation} {combined_csv}

/Library/Frameworks/Python.framework/Versions/3.9/bin/python3: can't open file '/home/yihuang/Documents/Data/covid19-forecast-hub/code/validation/validate_single_forecast_file.py': [Errno 2] No such file or directory
